<a href="https://colab.research.google.com/github/lagom-QB/M11/blob/master/EMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# EMNIST

In [1]:
from torchvision import datasets, transforms
import torch

train_dataset = datasets.EMNIST('/data', train=True, download=True,
                                transform=transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.1307,), (0.3081,))
                                ]),
                                split="balanced")

test_dataset = datasets.EMNIST('../data', train=False, download=True,
                                transform=transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.1307,), (0.3081,))
                                ]),
                                split="balanced")

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=True)

Extracting /data/EMNIST/raw/emnist.zip to /data/EMNIST/raw
Processing byclass


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


Processing bymerge
Processing balanced
Processing letters
Processing digits
Processing mnist
Done!


#1

Find correspondence between classess and letters/digits

In [0]:
letters = train_dataset.classes_split_dict['letters']
digits  = train_dataset.classes_split_dict['digits']
byclass = train_dataset.classes_split_dict['byclass']  # if not why not
balanced = train_dataset.classes_split_dict['balanced']

In [18]:
len(balanced)

47

#2

Build and train a dense classifier for EMNIST dataset. Try to achieve the highest accuracy you can.\
Explain, what experiments have you perform.\
What optimizers have you tested?\
How network/optimizer parameters were chosen.

In [0]:
import torch.nn as nn

class Net(nn.Module):
    
    def __init__(self):
        
        super(Net, self).__init__()

        self.layer1 = nn.Sequential(
                        nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
                        nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(7 * 7 * 64, 1000)
        self.fc2 = nn.Linear(1000, 10)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [0]:
learning_rate = 0.001 # Small enough
num_epochs = 25 #If not why not
num_classes = len(balanced)
batch_size = 100

In [21]:
model = Net()

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
                    model.parameters(), 
                    lr=learning_rate)
scheduler = torch.optim.lr_scheduler(optimizer)

TypeError: ignored

In [0]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

EMNIST is an extention of MNIST dataset.
It has 47 classes (handwritten digits and leters), some of the letters are represented as two classes (upper and lower case) and some (ex. o, s) has only one class associated with them.

1. Find correspondence between classess and letters/digits.
2. Build and train a dense classifier for EMNIST dataset. Try to achieve the highest accuracy you can. Explain, what experiments have you perform. What optimizers have you tested? How network/optimizer parameters were chosen.
3. Train the same classifier on MNIST dataset (you will have to replace the last dense layer of your model).
4. Use EMNIST classifier to classify MNIST test dataset. To do this you'll need to restrict last layer to the labels which correspond to digits and choose the largest value (torch tensors can be indexed with lists, like this: `torch.tensor([[1,2],[3,4], [5, 6]])[[0, 1]]`). Which of the models performs better?
5. Present a short (2-3 pages long) report on this experiment. What approaches have you tried? How can you explain the results you got? Include tensorboard plots in the experiment report.
